## Ai Travel Agent [Made By *Ahmed Mujtaba*]

In [3]:
# import plotly.express as px
# fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
# fig.write_html('first_figure.html', auto_open=True)

In [1]:
%pip install openai plotly mapbox

   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   ----------- ---------------------------- 41.0/139.3 kB 2.0 MB/s eta 0:00:01
   ----------------- ---------------------- 61.4/139.3 kB 1.1 MB/s eta 0:00:01
   ------------------------------ ------- 112.6/139.3 kB 819.2 kB/s eta 0:00:01
   -------------------------------------- 139.3/139.3 kB 827.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.9 MB 2.3 MB/s eta 0:00:06
   ---------------------------------------- 0.1/11.9 MB 2.2 MB/s eta 0:00:06
    --------------------------------------- 0.2/11.9 MB 1.5 MB/s eta 0:00:08
    --------------------------------------- 0.2/11.9 MB 1.5 MB/s eta 0:00:08
    --------------------------------------- 0.3/11.9 MB 1.2 MB/s eta 0:00:10
   - -------------------------------------- 0.3/11.9 MB 1.3 MB/s eta 0:00:10
   - -------------------------------------- 0.4/11.9 MB 1.2 MB/s eta 0:00:10
  

In [15]:
import openai, os
from openai import OpenAI
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())
client : OpenAI = OpenAI()
mapbox_token = os.environ['MAPBOX_ACCESS_TOKEN']

In [16]:
# Create the assistant with instructions and tools
assistant = client.beta.assistants.create(
    name="Travel Assistant Bot",
    instructions="I'm a travel assistant ready to help you plan your next adventure! Ask me about destinations, activities, trip planning tips, and more. I can also help you analyze travel documents and make calculations.",
    tools=["code-interpreter", "retrieval"],
    model="gpt-3.5-turbo-1106"
)

BadRequestError: Error code: 400 - {'error': {'message': '6 validation errors for Request\nbody -> tools -> 0\n  value is not a valid dict (type=type_error.dict)\nbody -> tools -> 0\n  value is not a valid dict (type=type_error.dict)\nbody -> tools -> 0\n  value is not a valid dict (type=type_error.dict)\nbody -> tools -> 1\n  value is not a valid dict (type=type_error.dict)\nbody -> tools -> 1\n  value is not a valid dict (type=type_error.dict)\nbody -> tools -> 1\n  value is not a valid dict (type=type_error.dict)', 'type': 'invalid_request_error', 'param': None, 'code': None}}